In [1]:
import sys
sys.path.insert(1, '../utils/')
import county_matcher as cm

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

#### Specify # of Bins Used in Dropdowns Across All Questions:

In [3]:
num_bins = 5 #value == 5 for first iteration of this project

#### Import County Information:

In [20]:
# Get county data:
county_df = pd.read_csv("../data/interim/county_matcher_data.csv.gz") # test using only terrain data

# Show:
county_df.head()

,FIPS,County,State,Total Population,Population Density (Per Sq. Mile),% Pop Density D,% Children Under 10,% Children Under 10 D,% Children 10 and Older,% Children 10 and Older D,...,Unity Churches,Unity of the Brethren,Universal Fellowship of Metropolitan Community Churches,Unnamed: 66,Vicariate for the Palestinian/Jordanian Orthodox Christian Communities,Vineyard USA,Wayne Trail Missionary Baptist Association,Wesleyan Church,Wisconsin Evangelical Lutheran Synod,Zoroastrian
0,12086,Miami-Dade County,Florida,2705528.0,1424.03,4.0,11.0,3.0,9.2,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36081,Queens County,New York,2270976.0,20887.52,5.0,11.0,3.0,8.7,2.0,...,NaN,NaN,NaN,0.08,NaN,NaN,NaN,NaN,NaN,NaN
2,34017,Hudson County,New Jersey,671923.0,14545.81,5.0,13.0,3.0,7.8,2.0,...,NaN,NaN,NaN,0.10,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,Aleutians West Census Area,Alaska,5708.0,1.30,1.0,10.0,2.0,7.6,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6085,Santa Clara County,California,1924379.0,1490.51,4.0,12.0,3.0,10.0,2.0,...,NaN,NaN,0.02,0.07,NaN,NaN,NaN,NaN,0.08,NaN


In [21]:
# Bin Counties Based on Values:
county_binned_df = county_df.copy()
county_binned_labeled_df = county_df.copy()
for col in county_binned_df.columns[3:]:
    try:
        county_binned_df[col] = pd.cut(county_binned_df[col],bins=num_bins,labels=False)
        county_binned_labeled_df[col] = pd.cut(county_df[col],bins=num_bins)
    except:
        pass
county_df = county_binned_df

# Show:
county_df.head()

,FIPS,County,State,Total Population,Population Density (Per Sq. Mile),% Pop Density D,% Children Under 10,% Children Under 10 D,% Children 10 and Older,% Children 10 and Older D,...,Unity Churches,Unity of the Brethren,Universal Fellowship of Metropolitan Community Churches,Unnamed: 66,Vicariate for the Palestinian/Jordanian Orthodox Christian Communities,Vineyard USA,Wayne Trail Missionary Baptist Association,Wesleyan Church,Wisconsin Evangelical Lutheran Synod,Zoroastrian
0,12086,Miami-Dade County,Florida,1,0,3,2,2,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36081,Queens County,New York,1,1,4,2,2,1,1,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,34017,Hudson County,New Jersey,0,1,4,2,2,1,1,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,Aleutians West Census Area,Alaska,0,0,0,1,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6085,Santa Clara County,California,0,0,3,2,2,2,1,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN


In [50]:
# get_labels:
bin_data_lst = []
for col in county_binned_labeled_df.iloc[:,3:]:
    interval_lst = list(county_binned_labeled_df[col].unique())
    interval_lst_formatted = [str(x) for x in interval_lst]
    bin_data_lst.append(interval_lst_formatted)
bin_df = pd.DataFrame(bin_data_lst,index=county_binned_labeled_df.columns[3:]).T
bin_df = bin_df.iloc[:num_bins,:]
# Show bin edge values::
bin_df


,Total Population,Population Density (Per Sq. Mile),% Pop Density D,% Children Under 10,% Children Under 10 D,% Children 10 and Older,% Children 10 and Older D,% Male,% Male: Under 5 Years,% Male: 5 to 9 Years,...,Unity Churches,Unity of the Brethren,Universal Fellowship of Metropolitan Community Churches,Unnamed: 66,Vicariate for the Palestinian/Jordanian Orthodox Christian Communities,Vineyard USA,Wayne Trail Missionary Baptist Association,Wesleyan Church,Wisconsin Evangelical Lutheran Synod,Zoroastrian
0,"(2008230.0, 4016343.0]","(-71.867, 14381.51]","(3.4, 4.2]","(10.0, 15.0]","(2.6, 3.4]","(4.9, 9.8]","(1.8, 2.6]","(47.78, 53.56]","(2.46, 4.92]","(2.02, 4.04]",...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,"(-9923.565, 2008230.0]","(14381.51, 28762.98]","(4.2, 5.0]","(5.0, 10.0]","(1.8, 2.6]","(9.8, 14.7]","(2.6, 3.4]","(59.34, 65.12]","(-0.0123, 2.46]","(6.06, 8.08]",...,None,None,"(-0.00015, 0.03]","(-0.00857, 3.724]",None,"(0.07998, 0.08002]",None,"(0.00964, 0.082]","(-0.0233, 4.658]",None
2,"(8032569.0, 10040682.0]","(28762.98, 43144.45]","(0.996, 1.8]","(15.0, 20.0]","(3.4, 4.2]","(14.7, 19.6]","(3.4, 4.2]","(53.56, 59.34]","(4.92, 7.38]","(-0.0101, 2.02]",...,None,None,"(0.12, 0.15]","(3.724, 7.438]",None,None,None,"(0.298, 0.37]","(4.658, 9.316]",None
3,"(4016343.0, 6024456.0]","(57525.92, 71907.39]","(1.8, 2.6]","(-0.025, 5.0]","(0.996, 1.8]","(-0.0245, 4.9]","(0.996, 1.8]","(41.971, 47.78]","(9.84, 12.3]","(4.04, 6.06]",...,None,None,"(0.03, 0.06]","(7.438, 11.152]",None,None,None,"(0.082, 0.154]","(13.974, 18.632]",None
4,None,None,"(2.6, 3.4]","(20.0, 25.0]","(4.2, 5.0]","(19.6, 24.5]","(4.2, 5.0]","(65.12, 70.9]","(7.38, 9.84]","(8.08, 10.1]",...,None,None,"(0.09, 0.12]","(11.152, 14.866]",None,None,None,"(0.226, 0.298]","(9.316, 13.974]",None


#### Import User "Responses" and "Importances":

In [20]:
# Import data:
user_df = pd.read_excel("../data/external/user_responses/RDPM_Quiz_modified.xlsx",sheet_name="Upload")
question_feature_ref_df = pd.read_csv("../data/external/question_feature_ref.csv")
user_ref_df = user_df.merge(question_feature_ref_df,on="Question",how="left")
user_ref_df["max_importance"] = pd.read_excel("../data/external/user_responses/RDPM_Quiz_modified.xlsx",sheet_name="Dropdowns")["Importance"].max()

# Fill NA values with 0:
user_ref_df[['Response', 'Importance']] = user_ref_df[['Response', 'Importance']].fillna(value=0)
user_ref_df[["max_response"]] = num_bins #Note this should change if more bins exist for a question

# Special case for religion question:
try:
    user_ref_df.loc[user_ref_df["Question"]=="Rel-2)","county_column_ref"] = user_ref_df.loc[user_ref_df["Question"]=="Rel-2)"]["Response"].values[0]
    user_ref_df.loc[user_ref_df["Question"]=="Rel-2)","Response"] = num_bins
except:
    user_ref_df = user_ref_df[user_ref_df["county_column_ref"].notna()]

# Assign 0's to NA values in relevant religion columns in county
user_ref_df.head()

,Question,Measurement,Response,Importance,ILOC,county_column_ref,max_importance,max_response
0,Acs-1),0,0,0.0,0,Population Density (Per Sq. Mile),10.0,5
1,Acs-2),At least,2,10.0,1,% Children Under 10,10.0,5
2,Acs-3),0,5,10.0,2,% Children 10 and Older,10.0,5
3,Acs-4),Equal to,4,10.0,3,% Couples that are Same-Sex,10.0,5
4,Acs-5),0,1,10.0,4,% Population Over 25 with at Least a Bachelor ...,10.0,5


In [21]:
# For columns where the user selects "at_most..." or "at_least...", assign similar values for any counties matching the criteria:
at_least_cols = user_ref_df[user_df["Measurement"]=="At least"]["county_column_ref"].to_list()
at_least_vals = user_ref_df[user_df["Measurement"]=="At least"]["Response"].to_list()

for col_num in range(len(at_least_cols)):
    col_label = at_least_cols[col_num]
    county_df.loc[county_df[col_label]>=at_least_vals[col_num],col_label] = county_df[col_label].max()
    
at_most_cols = user_ref_df[user_df["Measurement"]=="At most"]["county_column_ref"].to_list()
at_most_vals = user_ref_df[user_df["Measurement"]=="At most"]["Response"].to_list()

for col_num in range(len(at_most_cols)):
    col_label = at_most_cols[col_num]
    county_df.loc[county_df[col_label]<=at_least_vals[col_num],col_label] = county_df[col_label].min()

In [22]:
# Relevant columns:
feature_cols = user_ref_df["county_column_ref"].to_list()

#### Find Closest Matching Counties:

In [23]:
n_neighbors=50 # number of neighbors to match
matched_counties= cm.county_matcher(county_df,user_ref_df,feature_cols,target_col="FIPS",n_neighbors = n_neighbors,metric="euclidean")

In [24]:
matched_counties

,FIPS,County,State,distance_metric
0,36005,Bronx County,New York,54.90
1,16059,Lemhi County,Idaho,54.95
2,54057,Mineral County,West Virginia,54.95
3,29125,Maries County,Missouri,55.00
4,45077,Pickens County,South Carolina,55.05
5,13111,Fannin County,Georgia,55.10
6,13281,Towns County,Georgia,55.10
7,37039,Cherokee County,North Carolina,55.10
8,13085,Dawson County,Georgia,55.10
9,26163,Wayne County,Michigan,55.15
